# 泰坦尼克号生存预测——模型选择与训练SVM与标准化KNN

LIJIANcoder97

##### 上一次作业已经使用KNN完成了模型训练，精度为0.72，但是还可以更高吗？我们知道KNN的预测方法依靠与大量的数据，所以如果数据之间单位差距过大，拟合效果就会差，本次使用SKlearn标准化的方法，对数据i进行标准化，再用KNN与SVM进行训练。

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

#### 调用sklearn LabelEncoder()，进行编码，文本转数字。

In [2]:
def encode_features(data_set, feature_names):
    for feature_name in feature_names:
        le = LabelEncoder()
        le.fit(data_set[feature_name])
        encoded_column = le.transform(data_set[feature_name])
        data_set[feature_name] = encoded_column
    return data_set

In [3]:
Taitan_train=pd.read_csv('dataset/titanic/train.csv')
Taitan_test=pd.read_csv('dataset/titanic/test.csv')

丢弃'Cabin','Name','Ticket'属性列

In [4]:
Taitan_train=Taitan_train.drop(['Cabin','Name','Ticket'],axis=1)
Taitan_test=Taitan_test.drop(['Cabin','Name','Ticket'],axis=1)

#### 对sex embark属性进行编码

In [5]:
le = LabelEncoder()
le.fit(Taitan_train['Sex'])
Taitan_train['Sex']=le.transform(Taitan_train['Sex']) 

In [6]:
le = LabelEncoder()
le.fit(Taitan_test['Sex'])
Taitan_test['Sex']=le.transform(Taitan_test['Sex']) 

#### Embarked 属性有缺失值，要先进行处理，缺失值替换为最多的登船口

In [7]:
Taitan_train['Embarked'] = Taitan_train['Embarked'].fillna('S')

In [8]:
l = LabelEncoder()
l.fit(Taitan_train['Embarked'])
l.classes_

array(['C', 'Q', 'S'], dtype=object)

In [9]:
Taitan_train['Embarked']=l.transform(Taitan_train['Embarked']) 

In [10]:
Taitan_test['Embarked'] = Taitan_test['Embarked'].fillna('S')
l = LabelEncoder()
l.fit(Taitan_test['Embarked'])
Taitan_test['Embarked']=l.transform(Taitan_test['Embarked']) 

#### 编码完毕
### 对Age 属性进行处理，缺失值替换

In [11]:
Taitan_train['Age'] = Taitan_train['Age'].fillna(Taitan_train['Age'].median())

In [12]:
Taitan_test['Age'] = Taitan_test['Age'].fillna(Taitan_test['Age'].median())

#### 创建family属性代表乘客的亲人数目探索其与存活率的关系

In [13]:
Taitan_train['Family']=Taitan_train['SibSp']+Taitan_train['Parch']+1

In [14]:
Taitan_test['Family']=Taitan_test['SibSp']+Taitan_test['Parch']+1

In [15]:
Taitan_test['Fare'] = Taitan_train['Fare'].fillna(Taitan_train['Fare'].median())

现在已经有了清理好的数据集了

In [16]:
Taitan_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family
0,1,0,3,1,22.0,1,0,7.2500,2,2
1,2,1,1,0,38.0,1,0,71.2833,0,2
2,3,1,3,0,26.0,0,0,7.9250,2,1
3,4,1,1,0,35.0,1,0,53.1000,2,2
4,5,0,3,1,35.0,0,0,8.0500,2,1


In [17]:
Taitan_resout=Taitan_test
Taitan_test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family
0,892,3,1,34.5,0,0,7.2500,1,1
1,893,3,0,47.0,1,0,71.2833,2,2
2,894,2,1,62.0,0,0,7.9250,1,1
3,895,3,1,27.0,0,0,53.1000,2,1
4,896,3,0,22.0,1,1,8.0500,2,3


In [18]:
Taitan_p=Taitan_train['Survived']
Taitan_train=Taitan_train.drop(['Survived'],axis=1)

# 数据标准化

In [19]:
from sklearn.preprocessing import StandardScaler
standa = StandardScaler()

In [20]:
standa.fit(Taitan_train)
Taitan_train=standa.transform(Taitan_train)

D:\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  


In [21]:
standa.fit(Taitan_test)
Taitan_test=standa.transform(Taitan_test)

D:\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  


# 模型训练
## KNN

In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

导入knn和准确率评估方法

首先通过网格搜索选出最合适的超参数，将训练集分为两部分一部分用于训练，一部分用于验证。

In [23]:
x_train=Taitan_train[0:712]
x_test=Taitan_train[712: ]
y_train=Taitan_p[0:712]
y_test=Taitan_p[712: ]

训练集分割完毕开始训练,使用网格搜索

In [24]:
for k in [2,3,4,5,6]:
    for p in [1,2]: ##1代表明可夫斯基距离，2代表欧拉距离
        for w in ['uniform','distance']: ##uniform 代表不考虑距离权重
            n= KNeighborsClassifier(n_neighbors=k,weights=w,p=p)
            n.fit(x_train,y_train)##构造模型！！
            y_p=n.predict(x_test)##预测
            print('k=',k,';p=',p,';w=',w,';结果:',classification_report(y_test,y_p))

k= 2 ;p= 1 ;w= uniform ;结果:               precision    recall  f1-score   support

           0       0.82      0.90      0.86       115
           1       0.78      0.66      0.71        64

   micro avg       0.81      0.81      0.81       179
   macro avg       0.80      0.78      0.79       179
weighted avg       0.81      0.81      0.81       179

k= 2 ;p= 1 ;w= distance ;结果:               precision    recall  f1-score   support

           0       0.86      0.78      0.82       115
           1       0.66      0.77      0.71        64

   micro avg       0.78      0.78      0.78       179
   macro avg       0.76      0.77      0.76       179
weighted avg       0.79      0.78      0.78       179

k= 2 ;p= 2 ;w= uniform ;结果:               precision    recall  f1-score   support

           0       0.82      0.90      0.86       115
           1       0.78      0.66      0.71        64

   micro avg       0.81      0.81      0.81       179
   macro avg       0.80      0.78      0.79

准确率的参考属性时micro

震惊！！，使用标准化数据准确路提升10个百分点，准确率最高为 0.84，分别是 5，2，uniform；5，2，distance；
选择5，2，distance

In [25]:
n= KNeighborsClassifier(n_neighbors=5,weights='distance',p=2)
n.fit(Taitan_train,Taitan_p)##构造模型！！
y_re=n.predict(Taitan_test)##预测

In [26]:
y_re

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,

In [27]:
Taitan_resout=pd.read_csv('dataset/titanic/gender_submission.csv')

In [28]:
Taitan_resout['Survived']=y_re

In [29]:
Taitan_re=Taitan_resout

In [30]:
Taitan_re.to_csv('dataset/titanic/RE-stknn.csv')

In [31]:
Taitan_re.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [32]:
Taitan_re.describe()

,PassengerId,Survived
count,418.000000,418.000000
mean,1100.500000,0.392344
std,120.810458,0.488858
min,892.000000,0.000000
25%,996.250000,0.000000
50%,1100.500000,0.000000
75%,1204.750000,1.000000
max,1309.000000,1.000000


# 分析

标准化数据knn的训练与预测完成了，理想情况下，测试集的预测结果准确率也会达到0.84，相比之前的0.72取得了巨大的提高。


下面将使用SVM模型进行预测

In [33]:
from sklearn.metrics import accuracy_score

In [34]:
from sklearn.svm import SVC

In [59]:
%%time
max=0
c_a=0
degree_a=0
kernel_a='a'
for c in range(1,100):
    for degree in range(3,20): 
        for kernel in ['rbf','poly','linear','sigmoid']: ##uniform 代表不考虑距离权重
           clf=SVC(C=c,degree=degree,kernel=kernel,gamma='auto')
           clf.fit(x_train,y_train) 
           y_p=clf.predict(x_test)
           a=accuracy_score(y_test,y_p,)
           if accuracy_score(y_test,y_p,)>max :
              max=a
              c_a=c
              degree_a=degree
              kernel_a=kernel

Wall time: 6min 29s


In [60]:
print('准确度',max,' c',c_a,' degree',degree_a,' kernel',kernel_a)

准确度 0.8715083798882681  c 3  degree 3  kernel rbf


In [63]:
clf=SVC(C=3,degree=3,kernel='rbf',gamma='auto')
clf.fit(Taitan_train,Taitan_p)
y_p=clf.predict(Taitan_test)

In [64]:
Taitan_resout['Survived']=y_p

In [65]:
Taitan_re=Taitan_resout

In [66]:
Taitan_re.to_csv('dataset/titanic/RE - SVM.csv')

In [68]:
Taitan_re.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


### NOW 我们已经用标准化得KNN 与SVM模型预测完毕了，准确率分别为0.84和0.87.